In [3]:
from fastai.text.all import *
path = Path("data")
files = get_text_files(path, folders = ['train', 'test'])

In [10]:
txts = L(["Fourscore and seven years ago, our fathers brought forth upon this continent a new nation, conceived in liberty, and dedicated to the pursuit of happiness.",
          "No, you don't understand!"])
txt = txts[0]

In [21]:
tkn = Tokenizer(WordTokenizer())
toks = tkn(txt)
print(coll_repr(toks, 30))

(#31) ['xxbos','xxmaj','fourscore','and','seven','years','ago',',','our','fathers','brought','forth','upon','this','continent','a','new','nation',',','conceived','in','liberty',',','and','dedicated','to','the','pursuit','of','happiness'...]


In [15]:
toks_all = txts.map(tkn)
toks_all

(#2) [['xxbos', 'xxmaj', 'fourscore', 'and', 'seven', 'years', 'ago', ',', 'our', 'fathers', 'brought', 'forth', 'upon', 'this', 'continent', 'a', 'new', 'nation', ',', 'conceived', 'in', 'liberty', ',', 'and', 'dedicated', 'to', 'the', 'pursuit', 'of', 'happiness', '.'],['xxbos', 'xxmaj', 'no', ',', 'you', 'do', "n't", 'understand', '!']]

In [19]:
num = Numericalize(min_freq=1)
num.setup(toks_all)
print(coll_repr(num.vocab,20))

(#48) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj',',','and','fourscore','seven','years','ago','our','fathers','brought','forth','upon'...]


In [22]:
nums = num(toks)[:20]; nums

TensorText([ 2,  8, 11, 10, 12, 13, 14,  9, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         9, 25])

In [23]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj fourscore and seven years ago , our fathers brought forth upon this continent a new nation , conceived'

In [24]:
nums_all = toks_all.map(num)

In [27]:
dl = LMDataLoader(nums_all)

Creating the language model:

In [ ]:
get_files = partial(get_text_files, folders=['train', 'test'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_files, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

Fine-tuning the language model:

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)